# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łańcuchy znaków i wyrażenia regularne</center>

## Wprowadzenie

W każdym silniku baz danych istnieją mechanizmy do porównywania, dopasowywania i manipulowania ciągami znaków (*string*). 

Oprócz podstawowej funkcjonalności polegającej na odpowiedzi na pytanie „czy ten ciąg pasuje do tego wzoru?” w bazach danych istnieją operatory i funkcje do wyodrębniania, zastępowania i podziałów pasujących podciągów do wzorca podanego przez użytkownika.


Istnieją trzy osobne podejścia do dopasowywania wzorców zapewniane przez PostgreSQL:  
- LIKE/ILIKE, 
- SIMILAR TO (standard SQL:1999),  
- wyrażenia regularne w stylu POSIX. 

Ten dokument ma na celu przybliżenie podstawowych funkcji działania na ciągach znakowych w PostgreSQL.

## LIKE/ILIKE

Wyrażenie LIKE zwraca wartość *true*, jeśli ciąg znaków odpowiada dokładnie podanemu wzorcowi. ILIKE natomiast to klauzula umożliwiająca dopasowanie wzorca jednak nie zwraca ona uwagę na wielkość liter. Wyrażeniem przeciwnym jest NOT LIKE lub NOT ILIKE. Składnia tego wyrażenia to:

string (LIKE|ILIKE) wzorzec,
string NOT (LIKE|ILIKE) wzorzec

Przy tym typie klauzuli wzorce tworzymy przy użyciu dwóch operatorów:
- _ - zastępuje pojedynczy znak,
- % - zastępuje dowolną długość znaków.

#### Przykład:
|Wyrażenie|Wynik|
|---|---:|
|'abc' LIKE 'abc'|   true|  
|'abc' LIKE 'a%'|    true|  
|'abc' LIKE '_b_'|   true|  
|'abc' LIKE 'c'   |   false|  
|'ABC' ILIKE 'abc'|   true | 
|'ABC' LIKE 'abc'|    false| 

Klauzule LIKE/ILIKE można zastąpić operatorami:
- ~~ równoważny do LIKE
- ~~\*  równoważny do ILIKE  
- !~~  równoważny do LIKE  
- !~~\* równoważny do NOT ILIKE

## SIMILAR TO
Klauzula SIMILAR TO tak samo jak LIKE/ILIKE zwraca wartość *true* lub *false* w zależności od tego, czy podany wzorzec pasuje do podanego ciągu. Różnica pomiędzy tymi operatorami polega na tym, że SIMILAR TO interpretuje wzorzec za pomocą definicji wyrażenia regularnego w standardzie SQL. Wyrażenia regularne SQL są połączeniem notacji LIKE i zwykłej notacji wyrażeń regularnych (POSIX).

Składnia tego zapytania ma postać:

string SIMILAR TO wzorzec,
string NOT SIMILAR TO wzorzec

Oprócz funkcji zapożyczonych z LIKE, SIMILAR TO obsługuje te metaznaki pasujące do wzorca zapożyczone z wyrażeń regularnych POSIX:

- \| - oznacza naprzemienność (jedną z dwóch alternatyw).
- \* - oznacza powtórzenie poprzedniego elementu zero lub więcej razy.
- \+ - oznacza powtórzenie poprzedniego elementu jeden lub więcej razy.
- \? - oznacza powtórzenie poprzedniego elementu zero lub jeden raz.
- {m} - oznacza powtórzenie poprzedniego elementu dokładnie m razy.
- {m,} - oznacza powtórzenie poprzedniego elementu m lub więcej razy.
- {m, n} - oznacza powtórzenie poprzedniego elementu co najmniej mi nie więcej niż n razy.

Nawiasów () można używać do grupowania elementów w jeden element logiczny. Wyrażenie w nawiasie \[...\] określa klasę znaków, podobnie jak w wyrażeniach regularnych POSIX.

|Wyrażenie|Wynik|
|---|---:|
|'abc' SIMILAR TO 'abc'|     true|
|'abc' SIMILAR TO 'a'|        false|
|'abc' SIMILAR TO '%(b\|d)%'| true|
|'abc' SIMILAR TO '(b\|c)%'|   false|

## Składnia POSIX

Wyrażenia regularne POSIX zapewniają więcej reguł tworzenia wzorców niż operatory LIKE i SIMILAR TO. Wiele narzędzi uniksowych, takich jak grep, sed lub awk, używa języka dopasowywania wzorców podobnego do tego używanego w PostgreSQL.

Wyrażenie regularne to opis ciągu znaków przy użyciu symboli w celu utworzenia wzorca ciągu, która umożliwia dopasowanie wzorców. Mówi się, że łańcuch pasuje do wyrażenia regularnego, jeśli jest on członkiem zbioru regularnego opisanego przez wyrażenie regularne. Podobnie jak w przypadku LIKE, znaki wzorcowe dokładnie pasują do znaków łańcuchowych, chyba że są znakami specjalnymi w języku wyrażeń regularnych - ale wyrażenia regularne używają innych znaków specjalnych niż LIKE. W przeciwieństwie do wzorców LIKE, wyrażenie regularne może pasować w dowolnym miejscu ciągu, chyba że wyrażenie regularne jest wyraźnie zakotwiczone na początku (^) lub na końcu łańcucha ($).

Używanie tego rodzaju dopasowania ciągu znaków odbywa się przez operatory:

- ~	pasuje do wzorca , wielkość liter ma znaczenie
- ~\*	pasuje do wzorca , wielkość liter nie ma znaczenie	
- !~	nie pasuje do wzorca , wielkość liter ma znaczenie	
- !~\*	nie pasuje do wzorca , wielkość liter nie ma znaczenie

#### Przykład:
|Wyrażenie|Wynik|
|---|---:|
|'abc' ~ 'abc'|    true|
|'abc' ~ '^a' |    true|
|'abc' ~ '(b\|d)'|  true|
|'abc' ~ '^(b\|c)'| false|

Język zapytań regularnych w PostgreSQL który będzie omawiany na zajęciach składa się z:

|Wzorzec|Dopasowanie|
|---|---|
|\* |sekwencja 0 lub więcej dopasowań atomu|
|\+ |sekwencja 1 lub więcej dopasowań atomu|
|\? |sekwencja 0 lub 1 dopasowań atomu
|{m.} |sekwencja dokładnie m dopasowań atomu|
|{m,} |sekwencja m lub więcej dopasowań atomu|
|{m, n}| sekwencja od m do n (włącznie) dopasowań atomu; m nie może przekraczać n|
|^| dopasuj od początku łańcucha znaków|
|$| dopasuj od końca łańcucha znaków|

Formularze używające {...} są znane jako granice. Liczby min w granicach są liczbami całkowitymi dziesiętnymi bez znaku z dopuszczalnymi wartościami od 0 do 255 włącznie.

Pełny opis omawianych funkcjonalności dostępny jest w [dokumentacji PostgreSQL](https://www.postgresql.org/docs/9.3/functions-matching.html) 

## Wybrane funkcje działające na znakach

Poza funkcją dopasowania w PostgreSQL istnieje również szereg funkcji predefiniowanych niepozwalających działanie ciągach znaków. W poniższej tabeli przedstawiono wybrane funkcje:

|Funkcja| Opis| Przykład| Wynik|
|:---|:---|:---:|---:|
|ASCII| Zwraca wartość kodu ASCII znaku lub punktu kodu Unicode znaku UTF8<br><br>| ASCII ('A')| 65|
|CHR |Konwertuj kod ASCII na znak lub punkt kodu Unicode na znak UTF8<br><br>|CHR (65) |'A'|
|CONCAT| Połączenie dwóch lub więcej ciągów w jeden<br><br>| CONCAT('A', 'B', 'C')| 'ABC'|
|CONCAT_WS| Połączenie ciągów znaków z separatorem<br><br>| CONCAT_WS(',', 'A', 'B', 'C')| 'A, B, C'|
|FORMAT| Łącznie ciągów zgodnie z zadanym wzorcem formatowania<br><br>| FORMAT('Witaj% s', 'PostgreSQL') |'Witaj PostgreSQL'|
|INITCAP| Konwertuj łańcuch znaków w styl nagłówka<br><br>| INITCAP('CZEść wAM') |"Cześć Wam"|
|LEFT| Zwraca pierwszy n znaku z lewej strony ciągu<br><br>| LEFT('ABC', 1) |'A'|
|LENGTH| Zwraca liczbę znaków w ciągu<br><br>| LENGTH('ABC')| 3|
|LOWER| Konwertuj ciąg na małe litery<br><br>|LOWER ('czEŚĆ wAM')| 'cześć wam'|
|LPAD| Uzupełnieni z lewej strony ciągu do zadanej długości zadanym ciągiem<br><br>|LPAD('123', 5, '00')| '00123'|
|LTRIM| Usuwanie najdłuższego ciąg zawierającego określone znaki z lewej strony ciągu wejściowego<br><br>| LTRIM ('00123')|'123'|
|MD5| Zwraca skrót MD5 ciągu szesnastkowego<br><br>| MD5('ABC')||
|POSITION| Zwraca lokalizację pod łańcucha w ciągu<br><br>| POSTION('B' w 'A B C') |3|
|REGEXP_MATCHES| Dopasuj wyrażenie regularne POSIX do łańcucha i zwraca pasujące podciągi<br><br>| REGEXP_MATCHES ('ABC', '^(A)(..)$', 'g');| {ABC}|
|REGEXP_REPLACE| Zamienia podciągi pasujące do wyrażenia regularnego POSIX na nowy podciąg<br><br>| REGEXP_REPLACE ('John Doe','(.*)(.*)’,’\2, \1′);| 'Doe, John'|
|REPEAT| Powtarza ciąg określoną liczbę razy<br><br>| REPEAT('\*', 5)| '\*\*\*\*\*'|
|REPLACE| Zamienia wszystkie wystąpienia w ciągu pod łańcucha z podciągu na zadany <br><br>REPLACE('ABC', 'B', 'A')| 'AAC'|
|REVERSE| Odwrócenie ciągu<br><br>| REVERSE ('ABC') 'CBA'|
|RIGHT| Zwraca ostatnie n znaków w ciągu. Kiedy n jest ujemne, zwracaj wszystkie oprócz pierwszego<br><br>| RIGHT('ABC', 2)| 'BC'|
|RPAD| Uzupełnieni z prawej strony ciągu do zadanej długości zadanym ciągiem<br><br>| RPAD('ABC', 6, 'xo') | 'ABCxox'|
|RTRIM| Usuwa najdłuższy ciąg zawierający określone znaki z prawej strony ciągu wejściowego<br><br>| RTRIM 'abcxxzx', 'xyz')|'abc'|
|SPLIT_PART| Dzieli ciąg na określonym ograniczniku i zwraca n-ty pod łańcuch<br><br>| SPLIT_PART('2017-12-31′, ’-', 2)| ’12’|
|SUBSTRING| Wyodrębnia podciąg z ciągu<br><br>| SUBSTRING('ABC', 1,1)| 'A'|
|TRIM| Usuwa najdłuższy ciąg zawierający określone znaki z lewej, prawej lub obu ciągów wejściowych<br><br>| TRIM('ABC')| 'ABC'|
|UPPER| Konwertuje ciąg na wielkie litery<br><br>|UPPER('CZEść wAM') |'CZEŚĆ WAM'|


## Zadania:
1. Znajdź wszystkie nazwy krajów rozpoczynających się na P.
2. Znajdź wszystkie nazwy krajów rozpoczynających się  P i kończących na s.
3. Znajdź wszystkie tytuły filmów, w których znajdują się cyfry.
4. Znajdź wszystkich pracowników z podwójnym imieniem lub nazwiskiem.
5. Znajdź wszystkie nazwiska aktorów rozpoczynających się od P lub C i mających 5 znaków.
6. Znajdź wszystkie tytuły filmów, w których występują słowa Trip lub Alone.
7. Przeanalizuj zapytania:
	- select first_name from actor where first_name ~ '^ Al\[a: z, 1: 9\] *'
	- select first_name from actor where first_name ~ * '^ al\[a: z, 1: 9\] *'


In [1]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from typing import Union, List, Tuple

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');


Zad 1

In [2]:
zad1 = "SELECT country.country FROM country WHERE country.country SIMILAR TO 'P%'"
pd.read_sql(zad1,con=connection)


,country
0,Pakistan
1,Paraguay
2,Peru
3,Philippines
4,Poland
5,Puerto Rico


Zad 2

In [3]:
zad2 = "SELECT country.country FROM country WHERE country.country SIMILAR TO 'P%s'"
pd.read_sql(zad2,con=connection)


,country
0,Philippines


Zad 3

In [4]:
zad3 = "SELECT film.title FROM film WHERE film.title SIMILAR TO '%(1|2|3|4|5|6|7|8|9|0)%'"
pd.read_sql(zad3,con=connection)
#zad3 = "SELECT film.title FROM film WHERE film.title SIMILAR TO '%(Moon|League)%'" 


,title


Zad 4

zawiera spacje - sprawdzenie

In [5]:
zad4_test_space = "SELECT country.country FROM country WHERE country.country SIMILAR TO '%(-|\s)%'"
pd.read_sql(zad4_test_space,con=connection)


,country
0,American Samoa
1,"Congo, The Democratic Republic of the"
2,Czech Republic
3,Dominican Republic
4,Faroe Islands
5,French Guiana
6,French Polynesia
7,Holy See (Vatican City State)
8,Hong Kong
9,New Zealand


zawiera "-"

In [6]:
zad4_test_2 = "SELECT category.name FROM category WHERE category.name SIMILAR TO '%(-|\s)%'"
pd.read_sql(zad4_test_2,con=connection)


,name
0,Sci-Fi


In [7]:
zad4= "SELECT staff.first_name, staff.last_name  FROM staff WHERE staff.first_name SIMILAR TO '%(-|\s)%' OR staff.last_name SIMILAR TO '%(-|\s)%'"
pd.read_sql(zad4,con=connection)


,first_name,last_name


Zad 5

In [8]:
zad5 = "SELECT actor.last_name FROM actor WHERE actor.last_name SIMILAR TO '(P|C)_{4}'"
pd.read_sql(zad5,con=connection)

,last_name
0,Chase
1,Posey
2,Pesci
3,Crowe
4,Chase
5,Close


Zad 6

In [9]:
zad6 = "SELECT film.title FROM film WHERE film.title SIMILAR TO '%(Trip|Alone)%'" 
pd.read_sql(zad6,con=connection)


,title
0,Alone Trip
1,Superfly Trip
2,Trip Newton
3,Varsity Trip


Zad 7

In [10]:
zad7_1 = "select first_name from actor where first_name ~ '^Al[a:z,1:9]*'"
pd.read_sql(zad7_1,con=connection)


,first_name
0,Alec
1,Albert
2,Albert
3,Al
4,Alan


In [11]:
zad7_2 = "select first_name from actor where first_name ~* '^al[a:z,1:9]*'"
pd.read_sql(zad7_2,con=connection)


,first_name
0,Alec
1,Albert
2,Albert
3,Al
4,Alan


Obserwacja nr 1) znaki zaczynają się od AL - w pierwszym przypadku sam znak ~ bez * oznacza, że wielkość liter ma znaczenie

In [12]:
zad7_1_test = "select first_name from actor where first_name ~ '^al[a:z,1:9]*'"
pd.read_sql(zad7_1_test,con=connection)

,first_name


Obserwacja nr 2 - znak * po elemencie oznacza, że może wystąpić 0 do wielu razy (w tym zero razy)

In [13]:
zad7_2_test = "select first_name from actor where first_name ~* '^a*l[a:z,1:9]*'"
pd.read_sql(zad7_2_test,con=connection)


,first_name
0,Lucille
1,Alec
2,Liza
3,Albert
4,Lucille
5,Albert
6,Laura
7,Al
8,Laurence
9,Alan


[a:z,1:9] wraz z * pozwala na dowolną ilość liter alabetu od a do z i liczb od 1 do 9 

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.


Screenshot od testów z test_main znajduje się w tym samym folderze, co ten raport, poniższe przykłady pokazują, jak funkcje działają z

In [14]:
import main

#main.film_title_case_insensitive(['BeD', 'BLOOD', 'SonS', 'son', 'cry'])
main.film_title_case_insensitive(['Giant', 'Harry', 'birdcage', 'Iron'])

,title
0,Birdcage Casper
1,Chocolat Harry
2,Giant Troopers
3,Harry Idaho
4,Hoosiers Birdcage
5,Iron Moon
6,Jaws Harry
7,Kentuckian Giant
8,Louisiana Harry
9,Princess Giant


In [15]:
main.film_cast('Birdcage Casper')

,first_name,last_name
0,Renee,Ball
1,Salma,Nolte
2,Kenneth,Pesci
3,Kenneth,Torn


In [16]:
main.film_in_category(1)

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
5,Berets Agent,English,Action
6,Bride Intrigue,English,Action
7,Bull Shawshank,English,Action
8,Caddyshack Jedi,English,Action
9,Campus Remember,English,Action


In [17]:
main.film_in_category('Action')

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
5,Berets Agent,English,Action
6,Bride Intrigue,English,Action
7,Bull Shawshank,English,Action
8,Caddyshack Jedi,English,Action
9,Campus Remember,English,Action


In [18]:
main.film_in_category_case_insensitive(1)

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
5,Berets Agent,English,Action
6,Bride Intrigue,English,Action
7,Bull Shawshank,English,Action
8,Caddyshack Jedi,English,Action
9,Campus Remember,English,Action


In [19]:
main.film_in_category_case_insensitive("acTion")

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
5,Berets Agent,English,Action
6,Bride Intrigue,English,Action
7,Bull Shawshank,English,Action
8,Caddyshack Jedi,English,Action
9,Campus Remember,English,Action


In [20]:
main.film_in_category_case_insensitive("Actio")

,title,languge,category


Gdy braknie litery, funkcja tak jak powinna, nie znajdzie wyników